In [1]:
from smolvla_in_isaac.dataset.loading import load_and_split_dataset, create_dataloaders
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
device

device(type='cuda')

In [2]:
repo_id = "eternalmay33/pick_place_test"

In [3]:
def make_delta_timestamps(delta_indices: list[int] | None, fps: int) -> list[float]:
    if delta_indices is None:
        return [0]
    return [i / fps for i in delta_indices]

delta_timestamps = {
    "action" : make_delta_timestamps(
        list(range(20)),
        30
    )
}

In [4]:
full_dataset, train_dataset, val_dataset = load_and_split_dataset(
    repo_id=repo_id, delta_timestamps=delta_timestamps
)

LeRobotDatasetMetadata({
    Repository ID: 'eternalmay33/pick_place_test',
    Total episodes: '39',
    Total frames: '8159',
    Features: '['action', 'observation.state', 'observation.images.front', 'observation.images.third_person', 'observation.images.gripper', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',



In [5]:
full_dataset.features

{'action': {'dtype': 'float32',
  'shape': (6,),
  'names': ['shoulder_pan.pos',
   'shoulder_lift.pos',
   'elbow_flex.pos',
   'wrist_flex.pos',
   'wrist_roll.pos',
   'gripper.pos']},
 'observation.state': {'dtype': 'float32',
  'shape': (6,),
  'names': ['shoulder_pan.pos',
   'shoulder_lift.pos',
   'elbow_flex.pos',
   'wrist_flex.pos',
   'wrist_roll.pos',
   'gripper.pos']},
 'observation.images.front': {'dtype': 'video',
  'shape': (480, 640, 3),
  'names': ['height', 'width', 'channels'],
  'video_info': {'video.height': 480,
   'video.width': 640,
   'video.codec': 'av1',
   'video.pix_fmt': 'yuv420p',
   'video.is_depth_map': False,
   'video.fps': 30.0,
   'video.channels': 3,
   'has_audio': False},
  'info': {'video.height': 480,
   'video.width': 640,
   'video.codec': 'av1',
   'video.pix_fmt': 'yuv420p',
   'video.is_depth_map': False,
   'video.fps': 30,
   'video.channels': 3,
   'has_audio': False}},
 'observation.images.third_person': {'dtype': 'video',
  'shape'

In [6]:
train_loader, val_loader = create_dataloaders(
    train_dataset, val_dataset
)

In [7]:
class MLP(nn.Module):
    def __init__(self, inputdim, size, actiondim):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=inputdim, out_features=size),
            nn.ReLU(),
            nn.Linear(size, size),
            nn.ReLU()
        )

        self.decoder = nn.Linear(size, actiondim)
    
    def forward(self, batch):
        batch = self.net(batch)
        batch = self.decoder(batch)

        return batch

In [8]:
model = MLP(6, 32, 6).to(device)

model

MLP(
  (net): Sequential(
    (0): Linear(in_features=6, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  )
  (decoder): Linear(in_features=32, out_features=6, bias=True)
)

In [9]:
lr = 1e-4
criterion = nn.MSELoss()
optimizer = Adam(params=model.parameters(), lr=lr)

In [14]:
loss_list = [] 
n_epoch = 3_000

model.train()
 
for itr in range(0, n_epoch+1):
    total_loss = 0
    b=0
    for batch in tqdm(train_loader):

        states = batch["observation.state"].to(device)

        actions = batch["action"].to(device)[:, 0, :]

        print("="*50)
        print(states[0])
        print(actions[0])

        # print(actions)

        # print(states)
        # print(actions)

        y_pred = model(states)
        loss   = criterion(y_pred, actions) 
        total_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        b += 1 
        
        loss_list.append(loss.item())
        
    print(f'Epoch {itr} Loss: {total_loss/b:.4f}')

  1%|          | 7/796 [00:00<00:49, 15.78it/s]

tensor([ -1.5393,  21.5249, -97.1096,  68.2947,  -1.6350,  20.5777],
       device='cuda:0')
tensor([ 3.3309,  1.6170, 23.8698, -6.2472,  2.5869,  1.8405], device='cuda:0')
tensor([  1.2953,  27.3674, -40.5030,  90.0032, -11.8048,  21.3686],
       device='cuda:0')
tensor([ 4.8853, -7.6596, 17.8119,  8.5512, -0.2352,  6.9785], device='cuda:0')
tensor([ -1.8946,  16.8034, -98.2341,  76.7019,   8.2281,  21.4283],
       device='cuda:0')
tensor([ 5.1814, -0.2553, 15.9132,  7.3992,  2.0643,  1.3804], device='cuda:0')
tensor([  9.5804, -25.1486, -38.8444,  45.5175,  18.7182,  20.8577],
       device='cuda:0')
tensor([ 7.0318, -1.3617, 21.1573,  0.9304,  1.0191,  1.8405], device='cuda:0')
tensor([ -7.8358,   9.4741, -61.8914,  95.9055,  -0.8280,  20.9605],
       device='cuda:0')
tensor([  4.8112, -11.8298,  18.6257,  25.4763,   0.9146,   1.9939],
       device='cuda:0')
tensor([13.0030, 10.1426, -4.7327, 67.6872, -2.6275, 32.5942], device='cuda:0')
tensor([ 1.7024, -6.1277, 23.3273,  2.8799

  2%|▏         | 15/796 [00:00<00:32, 24.33it/s]

tensor([ 11.9735,   2.2390, -17.8473,  81.9289,   0.4088,  20.8893],
       device='cuda:0')
tensor([ 11.8431,  -4.5957, -20.5244,  82.6318,   0.3919,   4.9847],
       device='cuda:0')
tensor([16.1963, 24.0647,  2.3340, 73.6565, 16.0294, 31.0620], device='cuda:0')
tensor([ -0.5922, -10.2979,  16.4557,  22.6407,   3.5276,  24.2331],
       device='cuda:0')
tensor([ -6.5913, -15.6493,  41.6720,  70.9859,  11.0637,  42.7840],
       device='cuda:0')
tensor([ 7.8460, -6.6383, 20.0723, 11.8299,  3.0572,  2.7607], device='cuda:0')
tensor([  2.0266,  37.2864, -56.4701,  95.5877,  -2.8785,  34.1089],
       device='cuda:0')
tensor([-0.3701, -8.4255, 21.1573, -4.6522,  0.3397,  1.7638], device='cuda:0')
tensor([ -0.2934,  53.3564, -54.2116,  97.8045, -17.2422,  38.1563],
       device='cuda:0')
tensor([ 5.6255, -7.7447, 16.6365, 10.8551,  1.2281,  4.6779], device='cuda:0')
tensor([ 1.4094, -1.4124,  8.1403, 55.2152,  0.2298, 12.4883], device='cuda:0')
tensor([ 1.7024, -6.1277, 23.3273,  2.8799

  3%|▎         | 23/796 [00:01<00:25, 30.54it/s]

...

 31%|███       | 244/796 [00:08<00:19, 28.21it/s]


tensor([-5.0931, -4.2139, -2.2265, 85.2428,  4.5602, 42.3491], device='cuda:0')
tensor([ 7.8460, -6.6383, 20.0723, 11.8299,  3.0572,  2.7607], device='cuda:0')
tensor([-1.7701, -8.6147, 17.6911, 34.6983,  5.5164, 24.2159], device='cuda:0')
tensor([ -0.5922, -10.2979,  16.4557,  22.6407,   3.5276,  24.2331],
       device='cuda:0')
tensor([  9.3496, -23.8487,  -2.6055,  65.6143,  18.6775,  21.0752],
       device='cuda:0')
tensor([ 7.0318, -1.3617, 21.1573,  0.9304,  1.0191,  1.8405], device='cuda:0')
tensor([ -6.6634, -17.4313,  41.6454,  72.0716,  10.3380,  42.7846],
       device='cuda:0')
tensor([ 7.8460, -6.6383, 20.0723, 11.8299,  3.0572,  2.7607], device='cuda:0')


KeyboardInterrupt: 

Exception in thread Thread-18 (_pin_memory_loop):
Traceback (most recent call last):
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/multiprocessing/queues.py", line 122, in get


    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/site-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/multiprocessing/resource_sharer.py", line 86, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/multiprocessing/connection.py", line 519, in Client
    c = SocketClient(address)
        ^^^^^^^^^^^^^^^^^^^^^
  File "/home/may33/miniconda3/envs/isaac/lib/python3.11/multiprocessing/connection.py", line

In [ ]:

        states = batch["observation.state"].to(device)

        actions = batch["action"].to(device)

        # print(states)
        # print(actions)

        y_pred = model(states)
        loss   = criterion(y_pred, actions) 
        total_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        b += 1 
        
        loss_list.append(loss.item())
        
    print(f'Epoch {itr} Loss: {total_loss/b:.4f}')

In [ ]:
from pathlib import Path

modelpath = "/home/may33/projects/robotics/smolvla_in_isaac/training/checkpoints/mlpmodel.pth"

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'model_config': {
          'inputdim': 6,
          'size': 32,
          'actiondim': 6
      },
      'loss_list': loss_list,
      'n_epoch': n_epoch,
}, modelpath)

print(f"Model saved to {modelpath}")

Model saved to /home/may33/projects/robotics/smolvla_in_isaac/training/checkpoints/mlpmodel.pth
